# 4. 신경망 학습

* * *

+ <spen style="color: gold;">**학습**</spen> == 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것
+ <spen style="color: gold;">**손실함수**</spen> == 신경망이 학습할 수 있도록 해주는 <spen style="color: rosybrown;">**지표**</spen>  
<spen style="color: palevioletred;">**학습목표 : 손실 함수의 결과값을 가장 작게 만드는 가중치 매개변수를 찾는것**</spen>

## 4.4 기울기
+ $x_0$와 $x_1$의 편미분 동시에 계산
+ 기울기 ( gradient ) == 
( 
$
\frac{\partial f}{\partial x_0}
$,
$
\frac{\partial f}{\partial x_1}
$
 )
 처럼 모든 변수의 편미분을 벡터로 정리한 것

In [1]:
import numpy as np

# 편미분
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성 ( 원소가 모두 0인 배열 )

    # x의 각 원소에 대해서 수치미분
    for idx in range(x.size):
        tmp_val = x[idx]

        # f(x + h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)

        # f(x - h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2 * h) # 중앙 차분을 이용하여 기울기 계산
        x[idx] = tmp_val # 값 복원

    return grad

In [2]:
def function_2(x):
    # return x[0]**2 + x[1]**2
    return np.sum(x**2)

In [3]:
print("점 (3, 4) 기울기 =", numerical_gradient(function_2, np.array([3.0, 4.0])))
print("점 (0, 2) 기울기 =", numerical_gradient(function_2, np.array([0.0, 2.0])))
print("점 (3, 0) 기울기 =", numerical_gradient(function_2, np.array([3.0, 0.0])))

점 (3, 4) 기울기 = [6. 8.]
점 (0, 2) 기울기 = [0. 4.]
점 (3, 0) 기울기 = [6. 0.]


$
f(x_0, x_1) = x_0^2 + x_1^2
$
 의 기울기  
<img src="../img/gradient.png" width='600'>

+ 기울기의 결과에 마이너스를 붙인 벡터
+ 방향을 가진 벡터 ( 화살표 ) 로 그려짐
+ 기울기 == 함수의 '가장 낮은 장소 ( 최솟값 )'을 가리킴
    + 나침반처럼 화살표들은 한 점을 향하고 있음
+ '가장 낮은 곳'에서 멀어질수록 화살표의 크기가 커짐

<spen style="color: palevioletred;">**기울기가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향이다.**</spen>

##### <br><br><span style="color: mediumaquamarine;">**경사법 ( 경사 하강법 )**</spen>
+ 신경망은 최적 ( 손실 함수가 최솟값이 될 때의 매개변수 값 ) 의 매개변수 ( 가중치와 편향 ) 를 학습시에 찾아야함
+ <spen style="color: gold;">**경사하강법**</spen> == 기울기를 잘 이용해 함수의 최솟값 ( || 가능한 한 작은 값 ) 을 찾으려는 것
    + 사용 이유 : 매개변수 공간이 광대하여 어디가 최솟값이 되는 곳인지를 짐작할 수 없음

<spen style="color: rosybrown;">**주의점**</spen>  
+ 각 지점에서 함수의 값을 낮추는 방안을 제시하는 지표가 기울기라는 것  
    + 기울기가 가리키는 곳에 정말 함수의 최솟값이 있는지, 즉 그쪽이 정말로 나아갈 방향인지를 보장할 수 없음

안장점 ( 말 안장의 모양 )  
<img src="../img/saddle_point.png" width='300'>
+ 함수가 극솟값, 최솟값 || 안장점 이 되는 장소에서는 기울기가 0임
    + <spen style="color: gold;">**안장점 ( saddle point )**</spen> == 어느 방향에서 보면 극댓값이고 다른 방향에서 보면 극솟값이 되는 점

+ 복잡하고 찌그러진 모양의 함수라면 ( 대부분 ) 평평한 곳으로 파고들면서 고원 에 빠질 수 있음
    + <spen style="color: gold;">**고원 ( plateau )**</spen> == 학습이 진행되지 않는 정체기

+ 최솟값이 되는 장소를 찾는 문제 ( 아니면 가능한 한 작은 값이 되는 장소를 찾는 문제 ) 에서는 기울기 정보를 단서로 나아갈 방향을 정해야 함
    + 기울어진 방향이 꼭 최솟값을 가리키는 것은 아니나, 그 방향으로 가야 함수의 값을 줄일 수 있음
</br></br>
+ <spen style="color: gold;">**경사법 ( gradient method )**</spen> == 함수의 값을 점차 줄이는 것
    1. 현 위치에서 기울어진 방향으로 일정 거리만큼 이동
    2. 이동한 곳에서도 마찬가지로 기울기를 구하고, 기울어진 방향으로 나아가기를 반복
    + 경사법은 기계학습을 최적화하는 데 흔히 쓰는 방법
    + 특히 신경망 학습에는 경사법을 많이 사용

+ 경사법의 최솟값 == <spen style="color: gold;">**경사 하강법 ( Gradient Desence method )**</spen>
+ 경사법의 최댓값 == <spen style="color: gold;">**경사 상승법 ( Gradient Ascent method )**</spen>
+  <spen style="color: rosybrown;">**하강이냐 상승이냐는 본질적으로 중요하지 않음**</spen> ( 손실 함수의 부호를 반전시키면 최솟값을 찾는 문제와 최댓값을 찾는 문제는 같은것임 )
+ <spen style="color: palevioletred;">**일반적으로 신경망 ( 딥러닝 ) 분야에서의 경사법은 '경사 하강법'으로 등장할 때가 많음**</spen>

$$
x_0 := x_0 - \eta \frac{\partial f}{\partial x_0}
$$
$$
x_1 := x_1 - \eta \frac{\partial f}{\partial x_1}
$$

+ 변수가 2개인 경우의 1회 갱신
    + 변수의 수가 늘어도 같은 식 ( 각 변수의 편미분 값 ) 으로 갱신하게 됨
+ <spen style="color: gold;">**$\eta$ ( eta, 에타 )**</spen> == 갱신하는 양 == 신경망에서 학습률
    + <spen style="color: gold;">**학습률 ( learning rate )**</spen> == 한번의 학습으로 얼마만큼 학습해야 할지, 즉 매개변수 값을 얼마나 갱신하느냐를 정하는 것
        + 학습률 값은 0.01이나 0.001 등 미리 특정 값을 정해두어야함
        + 일반적으로 값이 너무 크거나 작으면 <spen style="color: rosybrown;">**'좋은 장소'**</spen>를 찾아갈 수 없음
        + 신경망 학습에서는 보통 학습률 값을 변경하면서 올바르게 학습하고 있는지를 확인하면서 진행
+ 번수의 값을 갱신하는 단계를 여러번 반복하면서 서서히 함수의 값을 줄임



In [4]:
# 경사 하강법 ( 함수의 극솟값을 구할 수 있고 잘하면 최솟값을 구할 수 있음 )
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x

    for i in range(step_num):
        grad = numerical_gradient(f, x) # 편미분 ( 함수의 기울기 )
        x -= lr * grad

    return x

+ ```f``` == 최적화하려는 함수
+ ```init_x``` == 초깃값
+ ```lr``` == 학습률 ( learning rate )
+ ```step_num``` == 경사법에 따른 반복 횟수
+ 편미분하여 구한 함수의 기울기에 학습률을 곱한 값으로 갱신하는 처리를 ```step_num```번 반복

문제1. 경사법으로 
$
f(x_0, x_1) = x_0^2 + x_1^2
$
의 최솟값을 구하라

In [5]:
def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

경사법을 사용한 갱신 과정을 그림으로 그림  
경사법에 의한 
$
f(x_0, x_1) = x_0^2 + x_1^2
$
의 갱신 과성 : 점선은 함수의 등고선을 나타낸다.  
<img src="../img/gradient_desence.png" width='600'>

+ 값이 가장 낮은 장소인 원점에 점차 가까워지고 있음

In [6]:
# 학습률이 너무 크거나 작으면 좋은 결과를 얻을 수 없다.

# 학습률이 너무 큰 예 : lr = 10.0
init_x = np.array([-3.0, 4.0])
print("학습률이 너무 클 때 :", gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100))

# 학습률이 너무 작은 예 : lr = 1e-10
init_x = np.array([-3.0, 4.0])
print("학습률이 너무 작을 때 :", gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100))


학습률이 너무 클 때 : [-2.58983747e+13 -1.29524862e+12]
학습률이 너무 작을 때 : [-2.99999994  3.99999992]


+ 실험 결과
    + 학습률이 너무 크면 큰 값으로 발산
    + 학습률이 너무 작으면 거의 갱신되지 않음

+ 학습률 같은 매개변수를 하이퍼파라미터 라고 함
    + <spen style="color: gold;">**하이퍼파라미터 ( Hyper parameter : 초매개변수 )**</spen> == 사람이 직접 설정해야하는 매개변수  
    ⨂ 가중치 매개변수는 훈련 데이터와 학습 알고리즘에 의해서 '자동'으로 획득되는매개변수
+ 하이퍼파라미터들은 여러 후보 값 중에서 시험을 통해 가장 잘 학습하는 값을 찾는 과정을 거쳐야함

##### <br><br><span style="color: mediumaquamarine;">**신경망에서의 기울기**</spen>

+ 가중치 매개변수에 대한 손실 함수의 기울기

가중치 == $W$, 손실 함수 == $L$인 신경망 ( 경사 == $\frac{\partial L}{\partial W}$)
$$
W = \begin{pmatrix}
w_{11} w_{12} w_{13}\\\\
w_{21} w_{22} w_{23}
\end{pmatrix}
$$

$$
\frac{\partial L}{\partial W} = \begin{pmatrix}
\frac{\partial L}{\partial W}_{11} \frac{\partial L}{\partial W}_{21} \frac{\partial L}{\partial W}_{31}\\\\
\frac{\partial L}{\partial W}_{21} \frac{\partial L}{\partial W}_{22} \frac{\partial L}{\partial W}_{23}
\end{pmatrix}
$$

+ $\frac{\partial L}{\partial W}$의 각 원소 == 각각의 원소에 관한 편미분  
ex ) 1행 1번째 원소인 $\frac{\partial L}{\partial W}_{11}$은 $W_11$을 조금 변경했을 때 손실함수 $L$이 얼마나 변화하느냐  
<spen style="color: rosybrown;">**중요점**</spen> : $\frac{\partial L}{\partial W}$의 형상이 $W$와 같음

In [4]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from fucntion import softmax, cross_entropy_error
from gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3) # 정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

+ 정규 분포 == 평균과 표준편차가 주어져 있을 때 엔트로피를 최대화하는 분포